In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
workdir_path = '/content/drive/My Drive/' + '00 PUC BI MASTER/00 - PROJ (TCC)/PREDIÇÃO ATIVIDADES/event2mind'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

IMPORTS NECESSÁRIOS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from nltk.tokenize import word_tokenize
import nltk.data
nltk.download('punkt')

import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Carrega DataSets

In [ ]:
datasetCompleto = pd.read_csv('event2MindClean.csv')
clusterIntencoes = pd.read_csv('clusterIntencoes.csv', sep = ',')

In [ ]:
def removeCaracteres(nomeColuna, dataSet):
  dataSet[nomeColuna]= dataSet[nomeColuna].astype(str)
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` ve", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` s", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("'", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("&", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace(",", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("  ", " ")
  
removeCaracteres('Xintent_2', datasetCompleto)

JUNTAR TODAS AS INTENÇOES NUMA ÚNICA LISTA


In [ ]:
#cluster_intents da base intencoesAll
clusterIntencoes['Xintent_2'] = clusterIntencoes['Xintent']
dtSetWithCluster = clusterIntencoes[["cluster_intents", "Xintent_2"]].join(datasetCompleto.set_index('Xintent_2'), on = 'Xintent_2', how = 'left')  


In [ ]:
dtSetWithCluster.head(5)

,cluster_intents,Xintent_2,Indice,Xintent,Xemotion,Event,Xsent,Osent,Event_2,Xemotion_2
0,4,advantage,773,to have an advantage,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'pleased '
0,4,advantage,773,to have an advantage,smug,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'smug '
0,4,advantage,773,to have an advantage,excited,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'excited '
0,4,advantage,8698,advantage,good,PersonX takes ___ of opportunities,4.0,5.0,"'personx, ', ',', 'tak, ', ',', ', ', ',', 'op...",'good '
0,4,advantage,16524,to have an advantage,in control,PersonX throws off balance,2.0,2.0,"'personx, ', ',', 'throw, ', ',', 'balanc, '",'control '


In [ ]:
dtSetWithCluster[dtSetWithCluster['cluster_intents'] == 5]

In [ ]:
#função para criar Polaridade
def polaridade_positiva(x):
  if x >= 4:
    return 1
  else:
    return 0

def polaridade_negativa(x):
  if x <= 2:
    return 1
  else:
    return 0

dtSetWithCluster['pos_polarity_Xsent'] = dtSetWithCluster['Xsent'].apply(polaridade_positiva) #com apply e função de classificação
dtSetWithCluster['neg_polarity_Xsent'] = dtSetWithCluster['Xsent'].apply(polaridade_negativa) #com apply e função de classificação

basePosit = dtSetWithCluster.groupby("cluster_intents")["pos_polarity_Xsent"].mean()
baseNeg = dtSetWithCluster.groupby("cluster_intents")["neg_polarity_Xsent"].mean()


In [ ]:
basePosit.head(10)

cluster_intents
0    0.661601
1    0.738486
2    0.733796
3    0.716212
4    0.741323
5    0.755194
6    0.657090
7    0.764458
Name: pos_polarity_Xsent, dtype: float64

In [ ]:
baseNeg.head(10)

cluster_intents
0    0.134633
1    0.094795
2    0.106289
3    0.115025
4    0.094316
5    0.082320
6    0.161326
7    0.101208
Name: neg_polarity_Xsent, dtype: float64

In [ ]:
dtSetWithCluster['pos_polarity_Osent'] = dtSetWithCluster['Osent'].apply(polaridade_positiva) #com apply e função de classificação
dtSetWithCluster['neg_polarity_Osent'] = dtSetWithCluster['Osent'].apply(polaridade_negativa) #com apply e função de classificação

basePosit = dtSetWithCluster.groupby("cluster_intents")["pos_polarity_Osent"].mean()
baseNeg = dtSetWithCluster.groupby("cluster_intents")["neg_polarity_Osent"].mean()



In [ ]:
basePosit.head(10)

cluster_intents
0    0.643847
1    0.722082
2    0.715278
3    0.694375
4    0.703471
5    0.731259
6    0.630571
7    0.740392
Name: pos_polarity_Osent, dtype: float64

In [ ]:
baseNeg.head(10)

cluster_intents
0    0.160995
1    0.122555
2    0.129051
3    0.145014
4    0.127389
5    0.110310
6    0.198343
7    0.126739
Name: neg_polarity_Osent, dtype: float64

In [ ]:
len(dtSetWithCluster['cluster_intents'].unique())

8

In [ ]:
dtSetWithCluster['cluster_intents'].unique()

array([4, 6, 5, 3, 2, 1, 7, 0])

In [ ]:
dtSetWithClusterDedup = dtSetWithCluster[['Xintent_2','cluster_intents']]

dtSetWithClusterDedup.drop_duplicates(keep=False, inplace=True)

dtSetWithClusterDedup.head()

,Xintent_2,cluster_intents
5,reform men bad attitude,2
10,instill fear people,7
11,people know important,5
12,others understand urgency,7
13,personal,2


FUNÇÃO PARA CRIAR CONTAGEM DE PALAVRAS POR CLUSTER

In [ ]:
def count_words_cluster(df, coluna_cluster):
  
  contagemClusters = pd.DataFrame(columns=['index', 'count', 'cluster'])
  
  num_cluster = len(df[coluna_cluster].unique())

  for i in range(0, num_cluster):
     
    filtro_cluster = df[df[coluna_cluster] == i] 
    #  ******* alterar a coluna Xintent_2 ******
    newdf = filtro_cluster.Xintent_2.str.split(expand=True).stack().value_counts().reset_index(name="count").query("count > 10") 
    newdf['cluster'] = i
    contagemClusters = contagemClusters.append(newdf)

  return contagemClusters

CHAMADA DA FUNÇÃO

In [ ]:
dtFrameContagens = count_words_cluster(dtSetWithClusterDedup, 'cluster_intents')

In [ ]:
dtFrameContagens.tail(5)

,index,count,cluster
80,light,11,7
81,conversation,11,7
82,success,11,7
83,gifts,11,7
84,assistance,11,7


In [ ]:
len(dtFrameContagens['cluster'].unique())

8

In [ ]:
dtFrameContagens['cluster'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=object)

In [ ]:
export_path = workdir_path + '/contagemClusterIntencoes.csv'

dtFrameContagens.to_csv (export_path, index = False, header=True)